In [ ]:
from pyspark.sql import SparkSession

In [ ]:
#dbutils.fs.unmount("/mnt/data")

/mnt/data has been unmounted.
Out[7]: True

In [ ]:
# Specify some variables to mount to blob storage
container_name = "####"
storage_name = "####"
mount_dir = "/mnt/data"
key = "####"


dbutils.fs.mount(
  source = "wasbs://%s@%s.blob.core.windows.net" %(container_name, storage_name),
  mount_point = mount_dir,
  extra_configs = {"fs.azure.account.key.%s.blob.core.windows.net" % (storage_name): key })


Out[8]: True

In [ ]:
# Create Spark Session
spark = SparkSession.builder.master('local').appName('app').getOrCreate()

# read parquet
df = spark.read.parquet("/mnt/data/cloud-storage-path/trade/trade_dt={}".format("2020-08-06"))

In [ ]:
df.show()

+----------+-----------+------+--------+--------------------+------------+--------------------+------+--------+
 trade_dt|record_type|symbol|exchange| event_tm|event_seq_nb| file_tm|bid_pr|bid_size|
+----------+-----------+------+--------+--------------------+------------+--------------------+------+--------+
2020-08-06| T| SYMB| NYSE|2020-08-06 16:57:...| 60|2020-08-06 09:30:...| 33| 773|
2020-08-06| T| SYMB| NYSE|2020-08-06 18:06:...| 70|2020-08-06 09:30:...| 34| 578|
2020-08-06| T| SYMB| NYSE|2020-08-06 19:21:...| 80|2020-08-06 09:30:...| 33| 718|
2020-08-06| T| SYMB| NYSE|2020-08-06 20:36:...| 90|2020-08-06 09:30:...| 33| 220|
2020-08-06| T| SYMB| NYSE|2020-08-06 21:46:...| 100|2020-08-06 09:30:...| 36| 741|
2020-08-06| T| SYMC| NYSE|2020-08-06 10:42:...| 10|2020-08-06 09:30:...| 157| 393|
2020-08-06| T| SYMC| NYSE|2020-08-06 11:52:...| 20|2020-08-06 09:30:...| 160| 954|
2020-08-06| T| SYMC| NYSE|2020-08-06 13:01:...| 30|2020-08-06 09:30:...| 158| 879|
2020-08-06| T| SYMC| NYSE|2020-08-06 14:15:...| 40|2020-08-06 09:30:...| 161| 232|
2020-08-06| T| SYMC| NYSE|2020-08-06 15:18:...| 50|2020-08-06 09:30:...| 158| 452|
2020-08-06| T| SYMC| NYSE|2020-08-06 16:24:...| 60|2020-08-06 09:30:...| 162| 912|
2020-08-06| T| SYMC| NYSE|2020-08-06 17:37:...| 70|2020-08-06 09:30:...| 161| 271|
2020-08-06| T| SYMC| NYSE|2020-08-06 19:00:...| 80|2020-08-06 09:30:...| 159| 931|
2020-08-06| T| SYMC| NYSE|2020-08-06 20:14:...| 90|2020-08-06 09:30:...| 158| 837|
2020-08-06| T| SYMC| NYSE|2020-08-06 21:32:...| 100|2020-08-06 09:30:...| 159| 516|
2020-08-05| T| SYMA| NYSE|2020-08-05 10:37:...| 10|2020-08-05 09:30:...| 79| 912|
2020-08-05| T| SYMA| NYSE|2020-08-05 11:56:...| 20|2020-08-05 09:30:...| 76| 962|
2020-08-05| T| SYMA| NYSE|2020-08-05 13:09:...| 30|2020-08-05 09:30:...| 75| 191|
2020-08-05| T| SYMA| NYSE|2020-08-05 14:24:...| 40|2020-08-05 09:30:...| 78| 53|
2020-08-05| T| SYMA| NYSE|2020-08-05 15:31:...| 50|2020-08-05 09:30:...| 78| 179|
+----------+-----------+------+--------+--------------------+------------+--------------------+------+--------+
only showing top 20 rows

In [ ]:
# Create a temp view for df
df.createOrReplaceTempView("trades")

# Create a new dataframe
temp_df = spark.sql("""
            SELECT trade_dt, 
                   record_type, 
                   symbol, 
                   exchange, 
                   event_tm, 
                   event_seq_nb,
                   bid_pr AS trade_pr
              FROM trades 
              WHERE trade_dt = '2020-08-06'

            """)

# Create a temp view from temp_df
temp_df.createOrReplaceTempView("tmp_trade_moving_avg")

In [ ]:
temp_df.show()

+----------+-----------+------+--------+--------------------+------------+--------+
 trade_dt|record_type|symbol|exchange| event_tm|event_seq_nb|trade_pr|
+----------+-----------+------+--------+--------------------+------------+--------+
2020-08-06| T| SYMB| NYSE|2020-08-06 16:57:...| 60| 33|
2020-08-06| T| SYMB| NYSE|2020-08-06 18:06:...| 70| 34|
2020-08-06| T| SYMB| NYSE|2020-08-06 19:21:...| 80| 33|
2020-08-06| T| SYMB| NYSE|2020-08-06 20:36:...| 90| 33|
2020-08-06| T| SYMB| NYSE|2020-08-06 21:46:...| 100| 36|
2020-08-06| T| SYMC| NYSE|2020-08-06 10:42:...| 10| 157|
2020-08-06| T| SYMC| NYSE|2020-08-06 11:52:...| 20| 160|
2020-08-06| T| SYMC| NYSE|2020-08-06 13:01:...| 30| 158|
2020-08-06| T| SYMC| NYSE|2020-08-06 14:15:...| 40| 161|
2020-08-06| T| SYMC| NYSE|2020-08-06 15:18:...| 50| 158|
2020-08-06| T| SYMC| NYSE|2020-08-06 16:24:...| 60| 162|
2020-08-06| T| SYMC| NYSE|2020-08-06 17:37:...| 70| 161|
2020-08-06| T| SYMC| NYSE|2020-08-06 19:00:...| 80| 159|
2020-08-06| T| SYMC| NYSE|2020-08-06 20:14:...| 90| 158|
2020-08-06| T| SYMC| NYSE|2020-08-06 21:32:...| 100| 159|
2020-08-06| T| SYMA| NYSE|2020-08-06 10:49:...| 10| 74|
2020-08-06| T| SYMA| NYSE|2020-08-06 12:00:...| 20| 76|
2020-08-06| T| SYMA| NYSE|2020-08-06 13:11:...| 30| 77|
2020-08-06| T| SYMA| NYSE|2020-08-06 14:27:...| 40| 77|
2020-08-06| T| SYMA| NYSE|2020-08-06 15:39:...| 50| 76|
+----------+-----------+------+--------+--------------------+------------+--------+
only showing top 20 rows

In [ ]:
# Calculate 30 mins moving average 
mov_avg_df = spark.sql("""
    SELECT 
           trade_dt, 
           record_type, 
           symbol, 
           exchange, 
           event_tm, 
           event_seq_nb,
           trade_pr,
           AVG(trade_pr) OVER (PARTITION BY (symbol)
                ORDER BY CAST(event_tm AS timestamp) 
                RANGE BETWEEN INTERVAL 30 MINUTES PRECEDING AND CURRENT ROW) as mov_avg_pr    
    FROM tmp_trade_moving_avg
""")

In [ ]:
# Save Temp View -> Hive Table for Staging
mov_avg_df.write.saveAsTable("temp_trade_moving_avg")

# Show
mov_avg_df.show(5)

+----------+-----------+------+--------+--------------------+------------+--------+----------+
 trade_dt|record_type|symbol|exchange| event_tm|event_seq_nb|trade_pr|mov_avg_pr|
+----------+-----------+------+--------+--------------------+------------+--------+----------+
2020-08-06| T| SYMA| NYSE|2020-08-06 10:49:...| 10| 74| 74.0000|
2020-08-06| T| SYMA| NYSE|2020-08-06 12:00:...| 20| 76| 76.0000|
2020-08-06| T| SYMA| NYSE|2020-08-06 13:11:...| 30| 77| 77.0000|
2020-08-06| T| SYMA| NYSE|2020-08-06 14:27:...| 40| 77| 77.0000|
2020-08-06| T| SYMA| NYSE|2020-08-06 15:39:...| 50| 76| 76.0000|
+----------+-----------+------+--------+--------------------+------------+--------+----------+
only showing top 5 rows

# Process Previous Trades Data

In [ ]:
prev_date_str = "2020-08-05"

# Create dateframe from prev_date_str
prev_df = spark.sql("select trade_dt, exchange, symbol, event_tm, event_seq_nb, bid_pr as trade_pr from trades where trade_dt = '{}'".format(prev_date_str))

# Create temp view
prev_df.createOrReplaceTempView("tmp_last_trade")

# Calculate last trade price 
last_prev_df = spark.sql("""
        SELECT a.symbol, a.exchange, a.last_pr FROM (
            SELECT symbol, exchange, event_tm, event_seq_nb, trade_pr,
            AVG(trade_pr) OVER (PARTITION BY (symbol)
            ORDER BY CAST(event_tm AS timestamp) 
            RANGE BETWEEN INTERVAL 30 MINUTES PRECEDING AND CURRENT ROW) as last_pr
             FROM tmp_trade_moving_avg
        ) a
""")

# Save Temp View Into Hive Table For Staging
last_prev_df.write.saveAsTable("temp_last_trade")

# Show head
last_prev_df.show(5)

+------+--------+-------+
symbol|exchange|last_pr|
+------+--------+-------+
 SYMA| NYSE|74.0000|
 SYMA| NYSE|76.0000|
 SYMA| NYSE|77.0000|
 SYMA| NYSE|77.0000|
 SYMA| NYSE|76.0000|
+------+--------+-------+
only showing top 5 rows

# Populate The Latest Trade and Latest Moving Average Trade Price To The Quote Records

In [ ]:
# Load the quote dataframe
quote_df = spark.read.parquet("/mnt/data/cloud-storage-path/quote/quote_dt={}".format("2020-08-06"))

# Create a temp view
quote_df.createOrReplaceTempView("quotes")

In [ ]:
quote_df.show(4)

+----------+-----------+------+--------+--------------------+------------+--------------------+------+--------+------+--------+
 trade_dt|record_type|symbol|exchange| event_tm|event_seq_nb| file_tm|bid_pr|bid_size|ask_pr|ask_size|
+----------+-----------+------+--------+--------------------+------------+--------------------+------+--------+------+--------+
2020-08-06| Q| SYMA| NASDAQ|2020-08-06 09:38:...| 1|2020-08-06 09:30:...| 78| 100| 80| 100|
2020-08-06| Q| SYMA| NASDAQ|2020-08-06 09:46:...| 2|2020-08-06 09:30:...| 77| 100| 77| 100|
2020-08-06| Q| SYMA| NASDAQ|2020-08-06 09:52:...| 3|2020-08-06 09:30:...| 79| 100| 79| 100|
2020-08-06| Q| SYMA| NASDAQ|2020-08-06 09:58:...| 4|2020-08-06 09:30:...| 76| 100| 77| 100|
+----------+-----------+------+--------+--------------------+------------+--------------------+------+--------+------+--------+
only showing top 4 rows

# Problem
Since we want to join `quotes` and `tmp_trade_moving_avg` to populate `trade_pr` and `mov_avg_pr` into quotes. However, we cannot use equality join in this case; trade events don't happen at the same quote time.

We want the latest in time sequence. This is a typical time sequence anatylical use case. A good method for this problem is to merge both tables in a common time sequence.

![Google Drive Image](https://drive.google.com/uc?export=view&id=1FNEf80Tw-lBb5JDkr1gDMmIPrgEn1plo)

In [ ]:
# Perform data normalization in the select statement from both table followed by union
quote_union = spark.sql("""
        SELECT  trade_dt,
                record_type,
                symbol,
                event_tm,
                event_seq_nb,
                exchange,
                bid_pr,
                bid_size,
                ask_pr,
                ask_size,
                null as trade_pr,
                null as mov_avg_pr 
            FROM quotes
            UNION ALL
        SELECT  trade_dt, 
                "T" as record_type,
                symbol,
                event_tm,
                event_seq_nb,
                exchange,
                null as bid_pr,
                null as bid_size,
                null as ask_pr,
                null as ask_size,
                trade_pr,
                mov_avg_pr 
             FROM temp_trade_moving_avg
""")

quote_union.show(10)

+----------+-----------+------+--------------------+------------+--------+------+--------+------+--------+--------+----------+
 trade_dt|record_type|symbol| event_tm|event_seq_nb|exchange|bid_pr|bid_size|ask_pr|ask_size|trade_pr|mov_avg_pr|
+----------+-----------+------+--------------------+------------+--------+------+--------+------+--------+--------+----------+
2020-08-06| Q| SYMA|2020-08-06 09:38:...| 1| NASDAQ| 78| 100| 80| 100| null| null|
2020-08-06| Q| SYMA|2020-08-06 09:46:...| 2| NASDAQ| 77| 100| 77| 100| null| null|
2020-08-06| Q| SYMA|2020-08-06 09:52:...| 3| NASDAQ| 79| 100| 79| 100| null| null|
2020-08-06| Q| SYMA|2020-08-06 09:58:...| 4| NASDAQ| 76| 100| 77| 100| null| null|
2020-08-06| Q| SYMA|2020-08-06 10:07:...| 5| NASDAQ| 77| 100| 79| 100| null| null|
2020-08-06| Q| SYMA|2020-08-06 10:15:...| 6| NASDAQ| 79| 100| 81| 100| null| null|
2020-08-06| Q| SYMA|2020-08-06 10:22:...| 7| NASDAQ| 78| 100| 79| 100| null| null|
2020-08-06| Q| SYMA|2020-08-06 10:29:...| 8| NASDAQ| 76| 100| 77| 100| null| null|
2020-08-06| Q| SYMA|2020-08-06 10:35:...| 9| NASDAQ| 76| 100| 78| 100| null| null|
2020-08-06| Q| SYMA|2020-08-06 10:50:...| 11| NASDAQ| 78| 100| 79| 100| null| null|
+----------+-----------+------+--------------------+------------+--------+------+--------+------+--------+--------+----------+
only showing top 10 rows

In [ ]:
# Create temp view
quote_union.createOrReplaceTempView("quote_union")

# Populate The Latest trade_pr and mov_avg_pr

In [ ]:
quote_union_update = spark.sql("""
    select *,
    last_value(trade_pr, true) OVER (PARTITION BY symbol, exchange ORDER BY event_tm ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as last_trade_pr,
    last_value(mov_avg_pr, true) OVER (PARTITION BY symbol, exchange ORDER BY event_tm ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as last_mov_avg_pr
    from quote_union
""")

quote_union_update.createOrReplaceTempView("quote_union_update")

In [ ]:
quote_union_update.show()

+----------+-----------+------+--------------------+------------+--------+------+--------+------+--------+--------+----------+-------------+---------------+
 trade_dt|record_type|symbol| event_tm|event_seq_nb|exchange|bid_pr|bid_size|ask_pr|ask_size|trade_pr|mov_avg_pr|last_trade_pr|last_mov_avg_pr|
+----------+-----------+------+--------------------+------------+--------+------+--------+------+--------+--------+----------+-------------+---------------+
2020-08-05| Q| SYMA|2020-08-05 09:36:...| 1| NASDAQ| 76| 100| 78| 100| null| null| null| null|
2020-08-05| Q| SYMA|2020-08-05 09:42:...| 2| NASDAQ| 75| 100| 76| 100| null| null| null| null|
2020-08-05| Q| SYMA|2020-08-05 09:48:...| 3| NASDAQ| 79| 100| 81| 100| null| null| null| null|
2020-08-05| Q| SYMA|2020-08-05 09:53:...| 4| NASDAQ| 75| 100| 76| 100| null| null| null| null|
2020-08-05| Q| SYMA|2020-08-05 10:00:...| 5| NASDAQ| 77| 100| 78| 100| null| null| null| null|
2020-08-05| Q| SYMA|2020-08-05 10:05:...| 6| NASDAQ| 77| 100| 78| 100| null| null| null| null|
2020-08-05| Q| SYMA|2020-08-05 10:13:...| 7| NASDAQ| 76| 100| 77| 100| null| null| null| null|
2020-08-05| Q| SYMA|2020-08-05 10:21:...| 8| NASDAQ| 77| 100| 78| 100| null| null| null| null|
2020-08-05| Q| SYMA|2020-08-05 10:30:...| 9| NASDAQ| 77| 100| 78| 100| null| null| null| null|
2020-08-05| Q| SYMA|2020-08-05 10:45:...| 11| NASDAQ| 78| 100| 79| 100| null| null| null| null|
2020-08-05| Q| SYMA|2020-08-05 10:54:...| 12| NASDAQ| 75| 100| 76| 100| null| null| null| null|
2020-08-05| Q| SYMA|2020-08-05 11:02:...| 13| NASDAQ| 78| 100| 79| 100| null| null| null| null|
2020-08-05| Q| SYMA|2020-08-05 11:11:...| 14| NASDAQ| 76| 100| 77| 100| null| null| null| null|
2020-08-05| Q| SYMA|2020-08-05 11:19:...| 15| NASDAQ| 79| 100| 80| 100| null| null| null| null|
2020-08-05| Q| SYMA|2020-08-05 11:29:...| 16| NASDAQ| 75| 100| 76| 100| null| null| null| null|
2020-08-05| Q| SYMA|2020-08-05 11:34:...| 17| NASDAQ| 78| 100| 78| 100| null| null| null| null|
2020-08-05| Q| SYMA|2020-08-05 11:41:...| 18| NASDAQ| 79| 100| 80| 100| null| null| null| null|
2020-08-05| Q| SYMA|2020-08-05 11:50:...| 19| NASDAQ| 79| 100| 80| 100| null| null| null| null|
2020-08-05| Q| SYMA|2020-08-05 12:03:...| 21| NASDAQ| 77| 100| 78| 100| null| null| null| null|
2020-08-05| Q| SYMA|2020-08-05 12:12:...| 22| NASDAQ| 75| 100| 76| 100| null| null| null| null|
+----------+-----------+------+--------------------+------------+--------+------+--------+------+--------+--------+----------+-------------+---------------+
only showing top 20 rows

In [ ]:
# Check the "T" - trade rows
spark.sql("""
        SELECT *
        FROM quote_union_update
        WHERE record_type = "T"
        
""").show()

+----------+-----------+------+--------------------+------------+--------+------+--------+------+--------+--------+----------+-------------+---------------+
 trade_dt|record_type|symbol| event_tm|event_seq_nb|exchange|bid_pr|bid_size|ask_pr|ask_size|trade_pr|mov_avg_pr|last_trade_pr|last_mov_avg_pr|
+----------+-----------+------+--------------------+------------+--------+------+--------+------+--------+--------+----------+-------------+---------------+
2020-08-06| T| SYMA|2020-08-06 10:49:...| 10| NYSE| null| null| null| null| 74| 74.0000| 74| 74.0000|
2020-08-06| T| SYMA|2020-08-06 12:00:...| 20| NYSE| null| null| null| null| 76| 76.0000| 76| 76.0000|
2020-08-06| T| SYMA|2020-08-06 13:11:...| 30| NYSE| null| null| null| null| 77| 77.0000| 77| 77.0000|
2020-08-06| T| SYMA|2020-08-06 14:27:...| 40| NYSE| null| null| null| null| 77| 77.0000| 77| 77.0000|
2020-08-06| T| SYMA|2020-08-06 15:39:...| 50| NYSE| null| null| null| null| 76| 76.0000| 76| 76.0000|
2020-08-06| T| SYMA|2020-08-06 16:58:...| 60| NYSE| null| null| null| null| 78| 78.0000| 78| 78.0000|
2020-08-06| T| SYMA|2020-08-06 18:14:...| 70| NYSE| null| null| null| null| 78| 78.0000| 78| 78.0000|
2020-08-06| T| SYMA|2020-08-06 19:28:...| 80| NYSE| null| null| null| null| 77| 77.0000| 77| 77.0000|
2020-08-06| T| SYMA|2020-08-06 20:49:...| 90| NYSE| null| null| null| null| 76| 76.0000| 76| 76.0000|
2020-08-06| T| SYMA|2020-08-06 22:00:...| 100| NYSE| null| null| null| null| 76| 76.0000| 76| 76.0000|
2020-08-06| T| SYMB|2020-08-06 10:45:...| 10| NYSE| null| null| null| null| 34| 34.0000| 34| 34.0000|
2020-08-06| T| SYMB|2020-08-06 12:01:...| 20| NYSE| null| null| null| null| 33| 33.0000| 33| 33.0000|
2020-08-06| T| SYMB|2020-08-06 13:13:...| 30| NYSE| null| null| null| null| 34| 34.0000| 34| 34.0000|
2020-08-06| T| SYMB|2020-08-06 14:29:...| 40| NYSE| null| null| null| null| 35| 35.0000| 35| 35.0000|
2020-08-06| T| SYMB|2020-08-06 15:46:...| 50| NYSE| null| null| null| null| 35| 35.0000| 35| 35.0000|
2020-08-06| T| SYMB|2020-08-06 16:57:...| 60| NYSE| null| null| null| null| 33| 33.0000| 33| 33.0000|
2020-08-06| T| SYMB|2020-08-06 18:06:...| 70| NYSE| null| null| null| null| 34| 34.0000| 34| 34.0000|
2020-08-06| T| SYMB|2020-08-06 19:21:...| 80| NYSE| null| null| null| null| 33| 33.0000| 33| 33.0000|
2020-08-06| T| SYMB|2020-08-06 20:36:...| 90| NYSE| null| null| null| null| 33| 33.0000| 33| 33.0000|
2020-08-06| T| SYMB|2020-08-06 21:46:...| 100| NYSE| null| null| null| null| 36| 36.0000| 36| 36.0000|
+----------+-----------+------+--------------------+------------+--------+------+--------+------+--------+--------+----------+-------------+---------------+
only showing top 20 rows

In [ ]:
# Filter For Quote Records
quote_update = spark.sql("""
    SELECT  trade_dt, 
            symbol, 
            event_tm, 
            event_seq_nb, 
            exchange,
            bid_pr, 
            bid_size, 
            ask_pr, 
            ask_size, 
            last_trade_pr, 
            last_mov_avg_pr
    FROM quote_union_update
    WHERE record_type = 'Q'
""")

# Create temporary view of data
quote_update.createOrReplaceTempView("quote_update")


# Join With Table temp_last_trade To Get The Prior Day Close Price
The prior day close price table has a single record per symbol and exchange. For this join, you can use equality join since the join fields are only “symbol” and “exchange”. However, note that this table has a very limited number of records (no more than the number of symbol and exchange combinations). This is an excellent opportunity to use broadcast join to achieve optimal join performance. Broadcast join is always recommended if one of the join tables is small enough so that the whole dataset fits in memory.

In [ ]:
quote_final = spark.sql("""
    SELECT trade_dt, 
           symbol, 
           event_tm, 
           event_seq_nb, 
           exchange,
           bid_pr, 
           bid_size, 
           ask_pr, 
           ask_size, 
           last_trade_pr, 
           last_mov_avg_pr,
           bid_pr - close_pr as bid_pr_mv,
           ask_pr - close_pr as ask_pr_mv
    FROM ( 
            SELECT /* + BROADCAST(t) */
                q.trade_dt, 
                q.symbol, 
                q.event_tm, 
                q.event_seq_nb, 
                q.exchange,
                q.bid_pr, 
                q.bid_size, 
                q.ask_pr, 
                q.ask_size, 
                q.last_trade_pr,
                q.last_mov_avg_pr,
                t.last_pr AS close_pr
            FROM quote_update q
             JOIN temp_last_trade t 
               ON (q.symbol = t.symbol AND q.exchange = t.exchange))
""")

# Show us the data so we can pick which trade_date to use
quote_final.show()

+----------+------+--------------------+------------+--------+------+--------+------+--------+-------------+---------------+---------+---------+
 trade_dt|symbol| event_tm|event_seq_nb|exchange|bid_pr|bid_size|ask_pr|ask_size|last_trade_pr|last_mov_avg_pr|bid_pr_mv|ask_pr_mv|
+----------+------+--------------------+------------+--------+------+--------+------+--------+-------------+---------------+---------+---------+
2020-08-05| SYMA|2020-08-05 09:34:...| 1| NYSE| 75| 100| 75| 100| null| null| -1.0000| -1.0000|
2020-08-05| SYMA|2020-08-05 09:34:...| 1| NYSE| 75| 100| 75| 100| null| null| -1.0000| -1.0000|
2020-08-05| SYMA|2020-08-05 09:34:...| 1| NYSE| 75| 100| 75| 100| null| null| -2.0000| -2.0000|
2020-08-05| SYMA|2020-08-05 09:34:...| 1| NYSE| 75| 100| 75| 100| null| null| -3.0000| -3.0000|
2020-08-05| SYMA|2020-08-05 09:34:...| 1| NYSE| 75| 100| 75| 100| null| null| -3.0000| -3.0000|
2020-08-05| SYMA|2020-08-05 09:34:...| 1| NYSE| 75| 100| 75| 100| null| null| -1.0000| -1.0000|
2020-08-05| SYMA|2020-08-05 09:34:...| 1| NYSE| 75| 100| 75| 100| null| null| -2.0000| -2.0000|
2020-08-05| SYMA|2020-08-05 09:34:...| 1| NYSE| 75| 100| 75| 100| null| null| -2.0000| -2.0000|
2020-08-05| SYMA|2020-08-05 09:34:...| 1| NYSE| 75| 100| 75| 100| null| null| -1.0000| -1.0000|
2020-08-05| SYMA|2020-08-05 09:34:...| 1| NYSE| 75| 100| 75| 100| null| null| 1.0000| 1.0000|
2020-08-05| SYMA|2020-08-05 09:40:...| 2| NYSE| 77| 100| 79| 100| null| null| 1.0000| 3.0000|
2020-08-05| SYMA|2020-08-05 09:40:...| 2| NYSE| 77| 100| 79| 100| null| null| 1.0000| 3.0000|
2020-08-05| SYMA|2020-08-05 09:40:...| 2| NYSE| 77| 100| 79| 100| null| null| 0.0000| 2.0000|
2020-08-05| SYMA|2020-08-05 09:40:...| 2| NYSE| 77| 100| 79| 100| null| null| -1.0000| 1.0000|
2020-08-05| SYMA|2020-08-05 09:40:...| 2| NYSE| 77| 100| 79| 100| null| null| -1.0000| 1.0000|
2020-08-05| SYMA|2020-08-05 09:40:...| 2| NYSE| 77| 100| 79| 100| null| null| 1.0000| 3.0000|
2020-08-05| SYMA|2020-08-05 09:40:...| 2| NYSE| 77| 100| 79| 100| null| null| 0.0000| 2.0000|
2020-08-05| SYMA|2020-08-05 09:40:...| 2| NYSE| 77| 100| 79| 100| null| null| 0.0000| 2.0000|
2020-08-05| SYMA|2020-08-05 09:40:...| 2| NYSE| 77| 100| 79| 100| null| null| 1.0000| 3.0000|
2020-08-05| SYMA|2020-08-05 09:40:...| 2| NYSE| 77| 100| 79| 100| null| null| 3.0000| 5.0000|
+----------+------+--------------------+------------+--------+------+--------+------+--------+-------------+---------------+---------+---------+
only showing top 20 rows

In [ ]:
# Write The Final Dataframe Into Azure Blob Storage At Corresponding Partition
trade_date = "2020-08-06"

quote_final.write.parquet("/mnt/data/cloud-storage-path/quote-trade-analytical/date={}".format(trade_date))


# END